In [1]:
import numpy as np
import pickle
import os

seed = 2023

In [2]:
from nlpsig_networks.scripts.ffn_baseline_functions import (
    histories_baseline_hyperparameter_search
)

In [3]:
output_dir = "therapist_talk_type_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## AnnoMI

In [4]:
%run ../load_anno_mi.py

In [5]:
anno_mi.head()

,mi_quality,transcript_id,topic,utterance_id,interlocutor,timestamp,utterance_text,annotator_id,therapist_input_exists,therapist_input_subtype,reflection_exists,reflection_subtype,question_exists,question_subtype,main_therapist_behaviour,client_talk_type,datetime
0,high,0,reducing alcohol consumption,0,therapist,00:00:13,Thanks for filling it out. We give this form t...,3,False,NaN,False,NaN,True,open,question,NaN,2023-07-27 00:00:13
1,high,0,reducing alcohol consumption,1,client,00:00:24,Sure.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-07-27 00:00:24
2,high,0,reducing alcohol consumption,2,therapist,00:00:25,"So, let's see. It looks that you put-- You dri...",3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-07-27 00:00:25
3,high,0,reducing alcohol consumption,3,client,00:00:34,Mm-hmm.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-07-27 00:00:34
4,high,0,reducing alcohol consumption,4,therapist,00:00:34,-and you usually have three to four drinks whe...,3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-07-27 00:00:34


In [6]:
with open("../anno_mi_sbert.pkl", "rb") as f:
    sbert_embeddings = pickle.load(f)
    
sbert_embeddings.shape

(13551, 384)

In [7]:
therapist_index

[True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 False,
 True,
 False,
 True,
 False,
 True,
 Fa

In [8]:
therapist_transcript_id

tensor([  0,   0,   0,  ..., 131, 131, 131])

# Baseline: FFN using signatures

First, we dimension reduce these and then take signatures. We use the path signature as input to the FFN for classification.

We want to choose a dimension and signature depth such that the number of terms in the signature is _roughly_ 384 so that it is comparable to the number of features that we used for the previous baseline where we computed the mean of the history. Again, we are concatenating the features we obtain with the current utterance embedding.

In [9]:
num_epochs = 100
hidden_dim_sizes = [[128,128],[256,256],[512,512]]
dropout_rates = [0.5, 0.2, 0.1]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [0, 1, 12, 123, 1234]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5

## Using log signature

In [10]:
log_signature_dimensions_and_sig_depths = [(28, 2), (10, 3), (6, 4)]

In [11]:
import signatory

[signatory.logsignature_channels(channels, depth)
 for (channels, depth) in log_signature_dimensions_and_sig_depths]

[406, 385, 406]

### Using UMAP

In [12]:
ffn_logsignature_umap_kfold, best_ffn_logsignature_umap_kfold, _, __ = histories_baseline_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="main_therapist_behaviour",
    embeddings=sbert_embeddings,
    y_data=y_data_therapist,
    output_dim=output_dim_therapist,
    hidden_dim_sizes=hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    use_signatures=True,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    log_signature=True,
    dim_reduce_methods=["umap"],
    dimension_and_sig_depths=log_signature_dimensions_and_sig_depths,
    path_indices=therapist_index,
    split_ids=therapist_transcript_id,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/ffn_logsignature_umap_focal_{gamma}_kfold.csv",
    verbose=False
)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/13551 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/13551 [00:00<?, ?it/s]

saving results dataframe to CSV for this hyperparameter search in therapist_talk_type_output/ffn_logsignature_umap_focal_2_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in therapist_talk_type_output/ffn_logsignature_umap_focal_2_kfold_best_model.csv


In [13]:
ffn_logsignature_umap_kfold.groupby(["dimension", "sig_depth", "hidden_dim", "dropout_rate", "learning_rate"]).mean()

/var/folders/l7/bnn1j6bs3bqfskq2jlq55yh80000gr/T/ipykernel_53974/3138393360.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ffn_logsignature_umap_kfold.groupby(["dimension", "sig_depth", "hidden_dim", "dropout_rate", "learning_rate"]).mean()


accuracy        f1  \
dimension sig_depth hidden_dim dropout_rate learning_rate                       
6         4         (128, 128) 0.1          0.0001         0.385488  0.349648   
                                            0.0005         0.399940  0.359578   
                                            0.0010         0.395565  0.355270   
                               0.2          0.0001         0.391432  0.351451   
                                            0.0005         0.406607  0.364229   
...                                                             ...       ...   
28        2         (512, 512) 0.2          0.0005         0.556705  0.531473   
                                            0.0010         0.553507  0.528184   
                               0.5          0.0001         0.558998  0.532636   
                                            0.0005         0.565364  0.537719   
                                            0.0010         0.552331  0.521688   

                                                           precision  \
dimension sig_depth hidden_dim dropout_rate learning_rate              
6         4         (128, 128) 0.1          0.0001          0.350472   
                                            0.0005          0.362823   
                                            0.0010          0.358817   
                               0.2          0.0001          0.353796   
                                            0.0005          0.368924   
...                                                              ...   
28        2         (512, 512) 0.2          0.0005          0.534058   
                                            0.0010          0.531857   
                               0.5          0.0001          0.534921   
                                            0.0005          0.539924   
                                            0.0010          0.526154   

                                                             recall  \
dimension sig_depth hidden_dim dropout_rate learning_rate             
6         4         (128, 128) 0.1          0.0001         0.354166   
                                            0.0005         0.364128   
                                            0.0010         0.358780   
                               0.2          0.0001         0.356168   
                                            0.0005         0.368157   
...                                                             ...   
28        2         (512, 512) 0.2          0.0005         0.534555   
                                            0.0010         0.529797   
                               0.5          0.0001         0.534628   
                                            0.0005         0.541377   
                                            0.0010         0.524966   

                                                           valid_accuracy  \
dimension sig_depth hidden_dim dropout_rate learning_rate                   
6         4         (128, 128) 0.1          0.0001               0.640538   
                                            0.0005               0.632738   
                                            0.0010               0.617978   
                               0.2          0.0001               0.624807   
                                            0.0005               0.605904   
...                                                                   ...   
28        2         (512, 512) 0.2          0.0005               0.707513   
                                            0.0010               0.692509   
                               0.5          0.0001               0.722736   
                                            0.0005               0.706081   
                                            0.0010               0.684314   

                                                           valid_f1  \
dimension sig_depth hidden_dim dropout_rate learning_rate             


In [14]:
best_ffn_logsignature_umap_kfold

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,loss_function,gamma,k_fold,n_splits,batch_size,input_dim,dimension,sig_depth,method,log_signature
0,None,0.563735,0.535800,"[0.5480354879594423, 0.3586597451628126, 0.476...",0.535847,"[0.5557840616966581, 0.32148900169204736, 0.50...",0.539116,"[0.5405, 0.4055496264674493, 0.448920023350846...",None,0.732540,...,focal,2,True,5,64,790,28,2,umap,True
0,None,0.567506,0.537299,"[0.5556869236226057, 0.35235732009925563, 0.46...",0.540395,"[0.5271422162404665, 0.3293135435992579, 0.507...",0.537892,"[0.5875, 0.37886872998932764, 0.42673671920607...",None,0.729566,...,focal,2,True,5,64,790,28,2,umap,True
0,None,0.550158,0.522643,"[0.5192540566723178, 0.32952853598014886, 0.46...",0.526138,"[0.5035227806481917, 0.3079777365491651, 0.482...",0.521232,"[0.536, 0.35432230522945574, 0.455341506129597...",None,0.725711,...,focal,2,True,5,64,790,28,2,umap,True
0,None,0.550309,0.526594,"[0.5241069198101423, 0.35545454545454547, 0.45...",0.529580,"[0.5237144283574638, 0.3095803642121932, 0.482...",0.528892,"[0.5245, 0.41728922091782283, 0.43199065966141...",None,0.732320,...,focal,2,True,5,64,790,28,2,umap,True
0,None,0.555740,0.532936,"[0.5250059822924145, 0.3744075829383886, 0.462...",0.536962,"[0.5034419458467186, 0.33674339300937767, 0.49...",0.533899,"[0.5485, 0.4215581643543223, 0.430823117338003...",None,0.725380,...,focal,2,True,5,64,790,28,2,umap,True


In [15]:
best_ffn_logsignature_umap_kfold["f1"].mean()

0.5310541080409561

In [16]:
best_ffn_logsignature_umap_kfold["precision"].mean()

0.5337843288286749

In [17]:
best_ffn_logsignature_umap_kfold["recall"].mean()

0.5322062593169047

In [18]:
np.stack(best_ffn_logsignature_umap_kfold["f1_scores"]).mean(axis=0)

array([0.53441787, 0.35408155, 0.46530654, 0.77041048])

In [19]:
np.stack(best_ffn_logsignature_umap_kfold["precision_scores"]).mean(axis=0)

array([0.52272109, 0.32102081, 0.49567492, 0.7957205 ])

In [20]:
np.stack(best_ffn_logsignature_umap_kfold["recall_scores"]).mean(axis=0)

array([0.5474    , 0.39551761, 0.43876241, 0.74714502])

### Using random projections

In [21]:
ffn_logsignature_grp_kfold, best_ffn_logsignature_grp_kfold, _, __ = histories_baseline_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="main_therapist_behaviour",
    embeddings=sbert_embeddings,
    y_data=y_data_therapist,
    output_dim=output_dim_therapist,
    hidden_dim_sizes=hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    use_signatures=True,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    log_signature=True,
    dim_reduce_methods=["gaussian_random_projection"],
    dimension_and_sig_depths=log_signature_dimensions_and_sig_depths,
    path_indices=therapist_index,
    split_ids=therapist_transcript_id,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/ffn_logsignature_grp_focal_{gamma}_kfold.csv",
    verbose=False
)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/13551 [00:00<?, ?it/s]

In [40]:
ffn_logsignature_grp_kfold.groupby(["dimension", "sig_depth", "hidden_dim", "dropout_rate", "learning_rate"]).mean()

/var/folders/l7/bnn1j6bs3bqfskq2jlq55yh80000gr/T/ipykernel_53974/2536377135.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ffn_logsignature_grp_kfold.groupby(["dimension", "sig_depth", "hidden_dim", "dropout_rate", "learning_rate"]).mean()


accuracy        f1  \
dimension sig_depth hidden_dim dropout_rate learning_rate                       
6         4         (128, 128) 0.1          0.0001         0.664806  0.638037   
                                            0.0005         0.660854  0.635033   
                                            0.0010         0.661518  0.634862   
                               0.2          0.0001         0.667914  0.641010   
                                            0.0005         0.664474  0.638683   
...                                                             ...       ...   
28        2         (512, 512) 0.2          0.0005         0.655604  0.629211   
                                            0.0010         0.656087  0.629127   
                               0.5          0.0001         0.660582  0.632962   
                                            0.0005         0.659496  0.631181   
                                            0.0010         0.659949  0.631869   

                                                           precision  \
dimension sig_depth hidden_dim dropout_rate learning_rate              
6         4         (128, 128) 0.1          0.0001          0.639940   
                                            0.0005          0.637411   
                                            0.0010          0.637938   
                               0.2          0.0001          0.643034   
                                            0.0005          0.640833   
...                                                              ...   
28        2         (512, 512) 0.2          0.0005          0.628320   
                                            0.0010          0.628296   
                               0.5          0.0001          0.632499   
                                            0.0005          0.630660   
                                            0.0010          0.631239   

                                                             recall  \
dimension sig_depth hidden_dim dropout_rate learning_rate             
6         4         (128, 128) 0.1          0.0001         0.639976   
                                            0.0005         0.637084   
                                            0.0010         0.636096   
                               0.2          0.0001         0.642754   
                                            0.0005         0.640276   
...                                                             ...   
28        2         (512, 512) 0.2          0.0005         0.632133   
                                            0.0010         0.631871   
                               0.5          0.0001         0.634008   
                                            0.0005         0.632602   
                                            0.0010         0.634474   

                                                           valid_accuracy  \
dimension sig_depth hidden_dim dropout_rate learning_rate                   
6         4         (128, 128) 0.1          0.0001               0.783675   
                                            0.0005               0.777704   
                                            0.0010               0.773408   
                               0.2          0.0001               0.785371   
                                            0.0005               0.780833   
...                                                                   ...   
28        2         (512, 512) 0.2          0.0005               0.776382   
                                            0.0010               0.770015   
                               0.5          0.0001               0.787729   
                                            0.0005               0.779863   
                                            0.0010               0.774664   

                                                           valid_f1  \
dimension sig_depth hidden_dim dropout_rate learning_rate             


In [33]:
best_ffn_logsignature_grp_kfold

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,loss_function,gamma,k_fold,n_splits,batch_size,input_dim,dimension,sig_depth,method,log_signature
0,None,0.665711,0.635158,"[0.6934875534322353, 0.47052280311457173, 0.57...",0.636587,"[0.697521497218007, 0.4912891986062718, 0.5746...",0.634396,"[0.6895, 0.45144076840981856, 0.57501459427904...",None,0.787178,...,focal,2,True,5,64,790,28,2,gaussian_random_projection,True
0,None,0.665108,0.636989,"[0.6986128625472887, 0.4807590933052187, 0.573...",0.636205,"[0.7048346055979644, 0.475, 0.5843446601941747...",0.638087,"[0.6925, 0.48665955176093917, 0.56217162872154...",None,0.790482,...,focal,2,True,5,64,790,28,2,gaussian_random_projection,True
0,None,0.665711,0.636030,"[0.6984126984126984, 0.46935568360398117, 0.56...",0.635073,"[0.7039106145251397, 0.4609053497942387, 0.575...",0.637251,"[0.693, 0.4781216648879402, 0.5557501459427904...",None,0.788940,...,focal,2,True,5,64,790,28,2,gaussian_random_projection,True
0,None,0.658169,0.630600,"[0.687888198757764, 0.46961038961038964, 0.571...",0.629797,"[0.7129828326180258, 0.4574898785425101, 0.570...",0.632141,"[0.6645, 0.4823906083244397, 0.572679509632224...",None,0.791584,...,focal,2,True,5,64,790,28,2,gaussian_random_projection,True
0,None,0.670840,0.642574,"[0.7000503271263211, 0.4844981607987389, 0.579...",0.641693,"[0.704660587639311, 0.4772256728778468, 0.5912...",0.643758,"[0.6955, 0.4919957310565635, 0.567425569176882...",None,0.789381,...,focal,2,True,5,64,790,28,2,gaussian_random_projection,True


In [34]:
best_ffn_logsignature_grp_kfold["f1"].mean()

0.6362703006106869

In [35]:
best_ffn_logsignature_grp_kfold["precision"].mean()

0.6358709901229604

In [36]:
best_ffn_logsignature_grp_kfold["recall"].mean()

0.6371265260022316

In [37]:
np.stack(best_ffn_logsignature_grp_kfold["f1_scores"]).mean(axis=0)

array([0.69569033, 0.47494923, 0.57276259, 0.80167906])

In [38]:
np.stack(best_ffn_logsignature_grp_kfold["precision_scores"]).mean(axis=0)

array([0.70478203, 0.47238202, 0.57917129, 0.78714862])

In [39]:
np.stack(best_ffn_logsignature_grp_kfold["recall_scores"]).mean(axis=0)

array([0.687     , 0.47812166, 0.56660829, 0.81677615])